In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm

In [2]:
%matplotlib widget

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

In [5]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [6]:
bdf_discharge = load_dir(dirname='/data/new_trap/20201116/Bead2/Discharge/', 
                         start_file=0, max_file=173)

2674  files in folder
173  files loaded


In [7]:
bandwidth = 2
decimate = 10
charges = 4
m3 = gfw.build_z_response(bdf_discharge[70:90], drive_freq=71, charges=charges, bandwidth=bandwidth, decimate=decimate, bimodal=False)

***************************************************
Z2-amplitude:  1.55e+03
reduced chi2:  7324.509804399726
***************************************************
Z2-amplitude:  1.55e+03
reduced chi2:  6150.4561388078855
***************************************************
Z2-amplitude:  1.55e+03
reduced chi2:  7788.471855384008
***************************************************
Z2-amplitude:  1.62e+03
reduced chi2:  5934.832109782137
***************************************************
Z2-amplitude:  1.51e+03
reduced chi2:  4679.594297134053
***************************************************
Z2-amplitude:  1.54e+03
reduced chi2:  5888.99520261955
***************************************************
Z2-amplitude:  1.53e+03
reduced chi2:  6771.66747199739
***************************************************
Z2-amplitude:  1.52e+03
reduced chi2:  7957.565796601954
***************************************************
Z2-amplitude:  1.58e+03
reduced chi2:  3999.0690011654624
*******************

In [8]:
scale = np.abs(np.mean([m_.values[0] for m_ in m3]))/charges
print("Scale is ", scale)

Scale is  385.61694118110563


In [10]:
ll = likelihood_analyser.LikelihoodAnalyser()
def func1_scale(i):
# for i,bdf_ in enumerate(data_bdfs2):
    print(i,'  ******************')
    fname = r'/data/new_trap/20201116/Bead2/Discharge/Neutrality_20201120/Discharge_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    drive_freq = int(bb.cant_freq)
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1000, 1000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    frequency = fit_kwargs['f']

    xx2 = bb.response_at_freq2('z', frequency, bandwidth=bandwidth)
    xx2 = xx2[5000:-5000:decimate]  # cut out the first and last second

    m1_tmp = ll.find_mle_sin(xx2, fsamp=500, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)


    return m1_tmp.values[0]

In [11]:
func1_scale(100)/scale

100   ******************


0.025646843197338128

In [37]:
res_tmp = np.array(Parallel(n_jobs=40)(delayed(func1_scale)(i) for i in tqdm(range(20000))))


 58%|█████▊    | 11600/20000 [07:55<06:29, 21.56it/s]


100%|██████████| 20000/20000 [14:03<00:00, 21.29it/s]

In [38]:
_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(res_tmp)), res_tmp/scale, label='10V, 71Hz., NoSpin')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
_,ax = plt.subplots()
data = res_tmp/scale
_ = ax.hist(data, bins=50, range=(-0.5,0.5))
ax.set(xlabel=r'$\epsilon$')
np.mean(data), np.std(data)/np.sqrt(len(data))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0012478821722029711, 0.0004081022287045877)

In [43]:
data = res_tmp/scale
means = [np.mean(data[j*1000:(j+1)*1000]) for j in range(20)]
stds = [np.std(data[j*1000:(j+1)*1000]) for j in range(20)]

In [44]:
_,ax = plt.subplots(figsize=(9.5,4))
ax.errorbar(np.arange(0,20000,1000), means, yerr=stds/np.sqrt(1000),fmt='o')
ax.plot(np.arange(0,20000,1000), np.arange(0,20000,1000)*0, 'r--')
ax.set(xlabel='dataset', ylabel=r'$\epsilon$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'dataset'), Text(0, 0.5, '$\\epsilon$')]

In [34]:
def get_Voltage(i):
    fname = r'/data/new_trap/20201116/Bead2/Discharge/Neutrality_20201120/Discharge_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    return bb.electrode_channels[0] #np.std(bb.electrode_data[0])*np.sqrt(2)

In [35]:
get_Voltage(10000)

1